In [1]:
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display
from sklearn import metrics
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_categorical_dtype
import numpy as np
import math
PATH = 'bluebook/'
df = pd.read_csv(f'{PATH}Train.csv',low_memory=False,parse_dates=['saledate'])
df

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401120,6333336,10500,1840702,21439,149,1.0,2005,NaN,NaN,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401121,6333337,11000,1830472,21439,149,1.0,2005,NaN,NaN,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401122,6333338,11500,1887659,21439,149,1.0,2005,NaN,NaN,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401123,6333341,9000,1903570,21435,149,2.0,2005,NaN,NaN,2011-10-25,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN


In [2]:
def apply_cats(df,train):
    for n,c in df.items():
        if train[n].datatype == 'category':
            df[n] = pd.Categorical(c,categories=train[n].cat.categories,ordered=True)
def train_cats(df):
    for n, c in df.items():
        if is_string_dtype(c):
            df[n]=c.astype('category').cat.as_ordered()
def add_datepart(df,dt_name,drop=True,time=False):
    dt_column=df[dt_name]
    column_dtype=dt_column.dtype
    attr=['year','month','week','day']
    for a in attr:
        df['Date' + a.capitalize()] = getattr(dt_column.dt,a)
    df['Date' + 'Elapsed']=dt_column.astype(np.int64)//10**9
    if drop:
        df.drop(dt_name,axis=1,inplace=True)
def numericalize(df,col,name):
    if not is_numeric_dtype(col) and is_categorical_dtype(col):
        df[name]=col.cat.codes +1

def fix_missing(df,col,name,nan_dict,is_train):
    if is_train:
        if is_numeric_dtype(col):
            if pd.isnull(col).any():
                df[name+'_na']=pd.isnull(col)
                nan_dict[name] = col.median()
                df[name] = col.fillna(nan_dict[name])
    else:
        if is_numeric_dtype(col):
            if name in nan_dict:
                df[name+'_na']=pd.isnull(col)
                df[name] = col.fillna(nan_dict[name])
            else:
                df[name] = col.fillna(df[name].median())
def proc_df(df,y_fld,nan_dict=None,is_train=True):
    df = df.copy()
    y=df[y_fld].values
    df.drop(y_fld,axis=1,inplace=True)
    
    if nan_dict is None:
        nan_dict={}
    for n,c in df.items():
        fix_missing(df,c,n,nan_dict,is_train)
        numericalize(df,c,n)
    if is_train:
        return df,y,nan_dict
    return df,y
def rmse(x,y):
    return math.sqrt(((x-y)**2).mean())
def split_train_val(df,n):
    return df[:n].copy(), df[n:].copy()

In [3]:
train_cats(df)
df = df.dropna(axis=1)
df

,SalesID,SalePrice,MachineID,ModelID,datasource,YearMade,saledate,fiModelDesc,fiBaseModel,fiProductClassDesc,state,ProductGroup,ProductGroupDesc
0,1139246,66000,999089,3157,121,2004,2006-11-16,521D,521,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,WL,Wheel Loader
1,1139248,57000,117657,77,121,1996,2004-03-26,950FII,950,Wheel Loader - 150.0 to 175.0 Horsepower,North Carolina,WL,Wheel Loader
2,1139249,10000,434808,7009,121,2001,2004-02-26,226,226,Skid Steer Loader - 1351.0 to 1601.0 Lb Operat...,New York,SSL,Skid Steer Loaders
3,1139251,38500,1026470,332,121,2001,2011-05-19,PC120-6E,PC120,"Hydraulic Excavator, Track - 12.0 to 14.0 Metr...",Texas,TEX,Track Excavators
4,1139253,11000,1057373,17311,121,2007,2009-07-23,S175,S175,Skid Steer Loader - 1601.0 to 1751.0 Lb Operat...,New York,SSL,Skid Steer Loaders
...,...,...,...,...,...,...,...,...,...,...,...,...,...
401120,6333336,10500,1840702,21439,149,2005,2011-11-02,35NX2,35,"Hydraulic Excavator, Track - 3.0 to 4.0 Metric...",Maryland,TEX,Track Excavators
401121,6333337,11000,1830472,21439,149,2005,2011-11-02,35NX2,35,"Hydraulic Excavator, Track - 3.0 to 4.0 Metric...",Maryland,TEX,Track Excavators
401122,6333338,11500,1887659,21439,149,2005,2011-11-02,35NX2,35,"Hydraulic Excavator, Track - 3.0 to 4.0 Metric...",Maryland,TEX,Track Excavators
401123,6333341,9000,1903570,21435,149,2005,2011-10-25,30NX,30,"Hydraulic Excavator, Track - 2.0 to 3.0 Metric...",Florida,TEX,Track Excavators


In [4]:
add_datepart(df,'saledate')

<ipython-input-2-0a3a7f1538b9>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date' + a.capitalize()] = getattr(dt_column.dt,a)
<ipython-input-2-0a3a7f1538b9>:14: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['Date' + a.capitalize()] = getattr(dt_column.dt,a)
<ipython-input-2-0a3a7f1538b9>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date' + 'Elapsed']=dt_column.astype(np.int64)//10**9
C:\Users\Aziz\anacond

In [5]:
n_valid = 12000
n_train = len(df)-n_valid
raw_train,raw_valid = split_train_val(df,n_train)
x_train, y_train, nas = proc_df(raw_train,'SalePrice')
x_valid, y_valid = proc_df(raw_valid,'SalePrice',nan_dict=nas,is_train=False)

In [6]:
from sklearn.model_selection import train_test_split

x, y, _ = proc_df(df,'SalePrice')

X_train, X_test, y_train, y_test = train_test_split(x,y ,random_state=104,test_size=0.25,shuffle=True)

In [7]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Random Forest Model

In [8]:
rfr = RandomForestRegressor(n_estimators=50,n_jobs=-1,min_samples_leaf=5,max_features=0.5,oob_score=True)
rfr.fit(X_train,y_train)
y_pred = rfr.predict(X_test)
print('R^2 Score:', rfr.score(X_test, y_test))

R^2 Score: 0.8953744506358504


# Support Vector Machine Model

In [ ]:
from sklearn import svm
svmreg = svm.SVR()
svmreg.fit(X_train,y_train)
y_pred = svmreg.predict(X_test)
print('R^2 Score:', svmreg.score(X_test, y_test))

# Decision Tree Regression

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeRegressor()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print('R^2 Score:', clf.score(X_test, y_test))